In [1]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, UpSampling2D, Flatten, BatchNormalization, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications.inception_v3 import preprocess_input
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import pickle
import cv2

In [2]:
X = pickle.load(open("X.pickle", "rb"))
y = pickle.load(open("y.pickle", "rb"))

In [3]:
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))
base_model.trainable = False
inputs = tf.keras.Input(shape=(299, 299, 3))
x = inputs

# pass results to a preprocessing layer
x = tf.keras.applications.inception_v3.preprocess_input(x)

# send results to the already trained model
x = base_model(x, training=False)

# send results to pooling layer
x = tf.keras.layers.GlobalAveragePooling2D()(x)

# send to fully connected layer
x = tf.keras.layers.Dense(256,activation='relu')(x)

# send to dropout layer
x = tf.keras.layers.Dropout(.5)(x)

# send to batch normalization layer
x = tf.keras.layers.BatchNormalization()(x)

# send to output dense layer with SoftMax activation function
outputs = tf.keras.layers.Dense(128,activation='softmax')(x)

# create a model with these layers
model = tf.keras.Model(inputs, outputs)
model.summary()

Metal device set to: Apple M1 Pro


2023-12-18 17:42:13.621438: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-12-18 17:42:13.622518: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 299, 299, 3)]     0         
                                                                 
 tf.math.truediv (TFOpLambda  (None, 299, 299, 3)      0         
 )                                                               
                                                                 
 tf.math.subtract (TFOpLambd  (None, 299, 299, 3)      0         
 a)                                                              
                                                                 
 inception_v3 (Functional)   (None, 8, 8, 2048)        21802784  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                             

In [4]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
x_train , x_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, stratify=y)

In [ ]:
model.fit(x_train, y_train, batch_size=32, epochs=10)

2023-12-18 17:42:46.079257: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/10


2023-12-18 17:42:49.474591: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


451/451 [==============================] - 95s 189ms/step - loss: 1.2524 - accuracy: 0.7571
Epoch 2/10
451/451 [==============================] - 83s 183ms/step - loss: 0.4655 - accuracy: 0.8797
Epoch 3/10
451/451 [==============================] - 83s 183ms/step - loss: 0.3776 - accuracy: 0.8927
Epoch 4/10
451/451 [==============================] - 83s 183ms/step - loss: 0.3412 - accuracy: 0.9017
Epoch 5/10
451/451 [==============================] - 82s 183ms/step - loss: 0.3191 - accuracy: 0.9027
Epoch 6/10
451/451 [==============================] - 83s 184ms/step - loss: 0.2944 - accuracy: 0.9108
Epoch 7/10
451/451 [==============================] - 83s 183ms/step - loss: 0.2679 - accuracy: 0.9159
Epoch 8/10
451/451 [==============================] - 82s 182ms/step - loss: 0.2453 - accuracy: 0.9233
Epoch 9/10
451/451 [==============================] - 83s 184ms/step - loss: 0.2349 - accuracy: 0.9270
Epoch 10/10
194/451 [===========>..................] - ETA: 46s - loss: 0.2325 - acc

In [ ]:
model.save('128dogs.h5')